In [1]:
import logging
from os import system
from glob import glob
from os.path import join
from pickle import dump, load
from datetime import datetime
from collections import namedtuple


import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import transformers
from transformers import Trainer, TrainingArguments
from transformers.integrations import TensorBoardCallback
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import AutoModel, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer, EarlyStoppingCallback, BatchEncoding
import optuna 

from arabert.preprocess import ArabertPreprocessor

import tensorboard_analysis 
from utilities import *

# Parameters

In [2]:
# Config
seed: int = 1

# Data 
data_proportion: float = 1.0 # propotion of data to be loaded in df
load_data: bool = True 
save_data: bool = False
test_validation_proportion: float = 0.02 # test and validation proportion from df

# Model 
# model_name = "aubmindlab/bert-base-arabertv2"
model_name: str = "aubmindlab/bert-base-arabertv02-twitter"
from_pretrained_classifier: bool = True
pretrained_classifier_name: str = "2021-12-09-train-0.963134765625"
    
# Preprocessing 
sequence_length: int = 32
tokenize_in_batches: bool = False # Helps reduce memory footprint

# Paths
code_folder_path: str = ""

# Training 
validation_size: int = 4096
batch_size: int = 64
learning_rate: float = 1e-5
epochs: int = 6
warmup_ratio: float = 0.1
save_model_while_training: bool = True # maybe doesn't work, transformers is terrible
do_warmup: bool = False
eval_while_training: bool = True # maybe doesn't work, transformers is terrible
save_model_after_finish: bool = True # maybe doesn't work, transformers is terrible

# Etc
open_tensorboard: bool = True

## Don't touch 

In [3]:
# Config
device = torch.device("cuda")
logger = logging.getLogger(__name__)
logger.setLevel(logging.ERROR)
logging.disable(logging.WARNING)

# General
date = str(datetime.today().date())

# Data
labels = ["EGY", "GLF", "IRQ", "LEV", "NOR"]

# Preprocessing
arabert_prep = ArabertPreprocessor(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Model 
if from_pretrained_classifier:
    pretrained_classifier_path = join(code_folder_path, "models", "finalized_models", pretrained_classifier_name)
    pretrained_classifier = AutoModelForSequenceClassification.from_pretrained(pretrained_classifier_path)

# Paths
train_path= f"./models/{date}-train"
search_path = f"./models/{date}-search"
dataset_string = "{}_dataset-seqlen" + str(sequence_length)


Downloading: 100%|██████████| 476/476 [00:00<00:00, 476kB/s]
Downloading: 100%|██████████| 733k/733k [00:02<00:00, 361kB/s]
Downloading: 100%|██████████| 1.19M/1.19M [00:03<00:00, 365kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 89.5kB/s]


In [4]:
assert 0 < data_proportion <= 1, "data_proportion must be right side inclusively between 0 and 1"
assert 0 <= warmup_ratio <= 1, "warmup_ratio must be inclusively between 0 and 1"
assert 0 < test_validation_proportion < 1, "test_validation_proportion must be exclusively between 0 and 1"
assert 0 < sequence_length, "sequence_length must be positive"
assert 0 < epochs, "epochs must be positive"
assert 0 < batch_size, "batch_size must be positive"
assert 0 < validation_size, "validation_size must be positive"


# Data (loading, preprocessing, tokenizing)

In [5]:
%%time
# Date to dataframe ~(2.9 s)
df = get_SMADC_folder_data(code_folder_path)
df = df.sample(frac=data_proportion)

# Encode Y ~(307 ms)
classes = df["Region"].unique()
num_labels = len(classes)
class_to_index = {class_:index for class_, index in zip(classes, range(len(classes)))}
index_to_class = {index:class_ for class_, index in zip(classes, range(len(classes)))}
df["Labels"] = df["Region"].apply(class_to_index.get)

Wall time: 5.31 s
Compiler : 121 ms


### Further preprocessing

In [6]:
if not load_data:
    # Preprocess X ~(16min 22s)
    df["Text"] = df["Text"].apply(arabert_prep.preprocess)

    # split and ~(323ms)
    train, test = train_test_split(df, test_size=test_validation_proportion, random_state=seed)
    validate, test = train_test_split(test, test_size=len(test)-validation_size, random_state=seed)
    train.reset_index(drop=True, inplace=True)
    validate.reset_index(drop=True, inplace=True)
    test.reset_index(drop=True, inplace=True)

    # Tokenize
    if tokenize_in_batches:
        validate_encoding = batch_tokenize(tokenizer, validate["Text"], 10, sequence_length)
        test_encoding = batch_tokenize(tokenizer, test["Text"], 100, sequence_length)
        train_encoding = batch_tokenize(tokenizer, train["Text"], 500, sequence_length)
    else:
        validate_encoding = tokenize(tokenizer, validate["Text"].to_list(), sequence_length)
        test_encoding = tokenize(tokenizer, test["Text"].to_list(), sequence_length)
        train_encoding = tokenize(tokenizer, list(train["Text"]), sequence_length)

    # Make Dataset 
    validate_dataset = Dialect_dataset(validate_encoding, validate["Labels"].to_list())
    test_dataset = Dialect_dataset(test_encoding, test["Labels"].to_list())
    train_dataset = Dialect_dataset(train_encoding, train["Labels"].to_list())

# Save & load preprocessed data

In [7]:
if save_data:
    save_preprocessed_data(validate_dataset, "preprocessed_validation")
    save_preprocessed_data(test_dataset, "preprocessed_test")
    save_preprocessed_data(train_dataset, "preprocessed_train")

if load_data:
    # ~(3mins)
    validate_dataset = load_preprocessed_data("preprocessed_validation")
    test_dataset = load_preprocessed_data("preprocessed_test")
    train_dataset = load_preprocessed_data("preprocessed_train")

# Training

In [8]:
training_args = generate_training_args(
                    train_path, epochs=epochs, do_warmup=do_warmup, warmup_ratio=warmup_ratio, 
                    save_model=save_model_while_training, eval_while_training=eval_while_training, 
                    learning_rate=learning_rate,batch_size=batch_size, 
                    train_dataset_length=len(train_dataset), seed=seed
                    )

# Begin train

In [9]:
if from_pretrained_classifier:
    trainer = Trainer(
    pretrained_classifier,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validate_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3), TensorBoardCallback()]
    )
else:
    trainer = Trainer(
        model_init=lambda:model_init(model_name, len(classes), label2id=class_to_index, id2label=index_to_class),
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validate_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3), TensorBoardCallback()]
    )

In [10]:
if open_tensorboard:
    from tensorboard import program
    tb = program.TensorBoard()
    tb.configure(argv=[None, '--logdir', join(code_folder_path, f"models")])
    print(f"Tensorflow listening on {tb.launch()}")

Tensorflow listening on http://localhost:6006/


In [11]:
class EvaluatePleaseCallback(transformers.TrainerCallback):
    def on_save(self, args, state, control, model, **kwargs):
        trainer.evaluate()
 
trainer.add_callback(EvaluatePleaseCallback())
trainer.train()

 15%|█▌        | 10000/64701 [1:29:06<6:04:16,  2.50it/s]

{'loss': 0.8545, 'learning_rate': 8.455974405341495e-06, 'epoch': 0.46}


 31%|███       | 20000/64701 [2:39:09<4:58:44,  2.49it/s]

{'loss': 0.6552, 'learning_rate': 6.911021467983494e-06, 'epoch': 0.93}


 46%|████▋     | 30000/64701 [4:07:23<14:00:16,  1.45s/it]

...

 93%|█████████▎| 60000/64701 [8:25:32<31:13,  2.51it/s]

{'loss': 0.5091, 'learning_rate': 7.304369329685785e-07, 'epoch': 2.78}


100%|██████████| 64701/64701 [8:56:45<00:00,  2.01it/s]

{'train_runtime': 32205.17, 'train_samples_per_second': 128.577, 'train_steps_per_second': 2.009, 'train_loss': 0.6062105185053168, 'epoch': 3.0}


TrainOutput(global_step=64701, training_loss=0.6062105185053168, metrics={'train_runtime': 32205.17, 'train_samples_per_second': 128.577, 'train_steps_per_second': 2.009, 'train_loss': 0.6062105185053168, 'epoch': 3.0})

# Analysis

In [12]:
trainer.evaluate(test_dataset)

100%|██████████| 377/377 [00:39<00:00,  9.47it/s]


{'eval_loss': 0.5220786333084106,
 'eval_macro_f1': 0.7832360568610907,
 'eval_macro_precision': 0.7910669284220544,
 'eval_macro_recall': 0.7766100128430244,
 'eval_accuracy': 0.8221317604054166,
 'eval_runtime': 40.2447,
 'eval_samples_per_second': 598.19,
 'eval_steps_per_second': 9.368,
 'epoch': 3.0}

# Save & load

In [13]:
if save_model_after_finish:
    trainer.save_model(f'models/finalized_models/{trainer.args.output_dir.split("/")[-1]}-{trainer.evaluate()["eval_accuracy"]}')

100%|██████████| 64/64 [00:06<00:00,  9.75it/s]


# Hyperparameter search

In [14]:
# def objective(trial: optuna.Trial):     
    
#     training_args = generate_training_args(search_path, epochs=None, do_warmup=False, save_model=False, eval_while_training=False)
#     training_args.learning_rate= trial.suggest_loguniform('learning_rate', low=4e-5, high=0.01)
#     training_args.weight_decay= trial.suggest_loguniform('weight_decay', 4e-5, 0.01)
#     training_args.num_train_epochs= trial.suggest_int('num_train_epochs', low=2, high=5)
    
#     trainer = Trainer(
#         model_init=lambda:model_init(model_name, len(classes), label2id=class_to_index, id2label=index_to_class),
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=validate_dataset,
#         compute_metrics=compute_metrics,
#         callbacks=[TensorBoardCallback()]
#     )
    
#     result = trainer.train()     
    
#     return result.training_loss # Or result.training_loss["metric_name"] ps: change direction in study if necessary
   
# We want to minimize the loss! 
# study = optuna.create_study(study_name='hyper-parameter-search', direction='minimize') 
# study.optimize(func=objective, n_trials=15)

# print(study.best_value) 
# print(study.best_params) 
# print(study.best_trial)

# trainer.hyperparameter_search(n_trials=10)
